<a href="https://colab.research.google.com/github/aditidagar/ocular_disease_recog/blob/main/PREPROCESSING/FUSION_WORK_OCULAR_DISEASE_PREPROCESSING_CLAHE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!echo '{"username":"arturusmaximus","key":"4f14194978499e9ae1ad6adb74b94add"}' > /content/kaggle.json
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k
!unzip ocular-disease-recognition-odir5k.zip
!mv ODIR-5K ODIR-5K_old
!mv ODIR-5K_old/ODIR-5K ODIR-5K
!mkdir ODIR-5K/Validation_Images


In [3]:
import tensorflow as tf
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import IPython.display as display
import numpy as np
import glob
import random

BATCH_SIZE = 32
CLASS_NAMES = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

# Set a random seed so we always use the same validation set (which is randomly sampled from dataset)
random.seed(7)

training_images_file = 'ODIR-5K/Training Images'
testing_images_file = 'ODIR-5K/Testing Images'
labels_file = 'ODIR-5K/data.xlsx'

diseases = {'abnormal pigment ': 'O', 'age-related macular degeneration': 'A', 'anterior segment image': 'DELETE', 'arteriosclerosis': 'O', 'asteroid hyalosis': 'O', 'atrophic change': 'O', 'atrophy': 'O', 'branch retinal artery occlusion': 'O', 'branch retinal vein occlusion': 'O', 'cataract': 'C', 'central retinal artery occlusion': 'O', 'central retinal vein occlusion': 'O', 'central serous chorioretinopathy': 'O', 'chorioretinal atrophy': 'O', 'chorioretinal atrophy with pigmentation proliferation': 'O', 'choroidal nevus': 'NaN', 'congenital choroidal coloboma': 'O', 'depigmentation of the retinal pigment epithelium': 'O', 'diabetic retinopathy': 'D', 'diffuse chorioretinal atrophy': 'O', 'diffuse retinal atrophy': 'O', 'drusen': 'O', 'dry age-related macular degeneration': 'A', 'epiretinal membrane': 'O', 'epiretinal membrane over the macula': 'O', 'fundus laser photocoagulation spots': 'O', 'glaucoma': 'G', 'glial remnants anterior to the optic disc': 'O', 'hypertensive retinopathy': 'H', 'hypertensive retinopathy,diabetic retinopathy': 'D', 'idiopathic choroidal neovascularization': 'O', 'image offset': 'DELETE', 'intraretinal hemorrhage': 'O', 'intraretinal microvascular abnormality': 'O', 'laser spot': 'O', 'lens dust': 'DELETE', 'low image quality': 'DELETE', 'low image quality,maculopathy': 'DELETE', 'macular coloboma': 'O', 'macular epiretinal membrane': 'O', 'macular hole': 'O', 'macular pigmentation disorder': 'NaN', 'maculopathy': 'O', 'mild nonproliferative retinopathy': 'D', 'moderate non proliferative retinopathy': 'D', 'morning glory syndrome': 'O', 'myelinated nerve fibers': 'O', 'myopia retinopathy': 'M', 'myopic maculopathy': 'M', 'myopic retinopathy': 'M', 'no fundus image': 'DELETE', 'normal fundus': 'N', 'old branch retinal vein occlusion': 'O', 'old central retinal vein occlusion': 'O', 'old chorioretinopathy': 'O', 'old choroiditis': 'O', 'optic disc edema': 'O', 'optic discitis': 'O', 'optic disk epiretinal membrane': 'O', 'optic disk photographically invisible': 'DELETE', 'optic nerve atrophy': 'O', 'oval yellow-white atrophy': 'O', 'pathological myopia': 'M', 'peripapillary atrophy': 'O', 'pigment epithelium proliferation': 'O', 'pigmentation disorder': 'O', 'post laser photocoagulation': 'O', 'post retinal laser surgery': 'O', 'proliferative diabetic retinopathy': 'D', 'punctate inner choroidopathy': 'O', 'refractive media opacity': 'O', 'retina fold': 'O', 'retinal artery macroaneurysm': 'O', 'retinal detachment': 'O', 'retinal pigment epithelial hypertrophy': 'O', 'retinal pigment epithelium atrophy': 'O', 'retinal pigmentation': 'O', 'retinal vascular sheathing': 'O', 'retinitis pigmentosa': 'O', 'retinochoroidal coloboma': 'O', 'rhegmatogenous retinal detachment': 'O', 'severe nonproliferative retinopathy': 'D', 'severe proliferative diabetic retinopathy': 'D', 'silicone oil eye': 'O', 'spotted membranous change': 'O', 'suspected abnormal color of  optic disc': 'O', 'suspected cataract': 'C', 'suspected diabetic retinopathy': 'D', 'suspected glaucoma': 'G', 'suspected macular epimacular membrane': 'O', 'suspected microvascular anomalies': 'O', 'suspected moderate non proliferative retinopathy': 'D', 'suspected retinal vascular sheathing': 'O', 'suspected retinitis pigmentosa': 'O', 'suspicious diabetic retinopathy': 'D', 'tessellated fundus': 'O', 'vascular loops': 'O', 'vessel tortuosity': 'O', 'vitreous degeneration': 'O', 'vitreous opacity': 'O', 'wedge white line change': 'O', 'wedge-shaped change': 'O', 'wet age-related macular degeneration': 'A', 'white vessel': 'O'}

#load labels to pandas
labels = pd.read_excel(labels_file, index_col=0) 
print(labels['Left-Diagnostic Keywords'][0])

cataract


Labeling Images by renaming them, removing low quality images 

In [4]:
file_path = 'ODIR-5K/Training Images'
all_paths = []
for element in glob.glob(file_path+"/*.jpg"):
  all_paths.append(element)
paths = []

# TODO: Count how many matching eye images?

#adding label information to image names
for u_id in labels.index:
  labelL = ""
  labelR = ""
  diagnosticL = labels['Left-Diagnostic Keywords'][u_id]
  diagnosticR = labels['Right-Diagnostic Keywords'][u_id]
  diagnosticL = diagnosticL.split("，") 
  diagnosticR = diagnosticR.split("，") 
  for d in diagnosticL:
    if d in diseases:
        if labelL != "":
          labelL+="&"+diseases[d]
        else:
          labelL+=diseases[d]
    else:
        labelL+="DELETE"
  for d in diagnosticR:
    if d in diseases:
      if labelR != "":
        labelR+="&"+diseases[d]
      else:
        labelR+=diseases[d]
    else:
      labelR+="DELETE"

  if file_path+"/"+str(labels['Left-Fundus'][u_id]) in all_paths:
    filename = str(labels['Left-Fundus'][u_id]).split(".")
    os.rename(file_path+"/"+str(labels['Left-Fundus'][u_id]), file_path+"/"+filename[0]+"-"+labelL+".jpg")
    paths.append(file_path+"/"+str(labels['Left-Fundus'][u_id])+"/"+labelL)

  if file_path+"/"+str(labels['Right-Fundus'][u_id]) in all_paths:
    filename = str(labels['Right-Fundus'][u_id]).split(".")
    os.rename(file_path+"/"+str(labels['Right-Fundus'][u_id]), file_path+"/"+filename[0]+"-"+labelR+".jpg")
    paths.append(file_path+"/"+str(labels['Right-Fundus'][u_id])+"/"+labelR)

#deleting low quality images    
items_to_remove = []
add_mix_info = []

for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if "DELETE" in img_label[-1]:
      items_to_remove.append(element)
    elif "&" in img_label[-1]:
      add_mix_info.append(element)

for e in items_to_remove:
  os.remove(e)

for e in add_mix_info:
  pom = e.split(".")
  os.rename(e, pom[0]+"&X"+".jpg")      

In [5]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

     |████████████████████████████████| 102 kB 5.7 MB/s 
     |████████████████████████████████| 47.6 MB 66 kB/s 
albumentations==1.1.0 is successfully installed


Image Enhancement and Resizing. Creation of Validation Set by Random Selection


In [6]:
import cv2 as cv
import albumentations as A

fusionMethod = "SUM"

transform = A.Compose([
   # A.CLAHE(always_apply=True, p=1.0, clip_limit=(1, 4), tile_grid_size=(43, 34))
   A.CLAHE(always_apply=True, p=1.0, clip_limit=(1, 2), tile_grid_size=(12, 6)),
   A.HueSaturationValue(always_apply=True, p=1.0, hue_shift_limit=(0, 0), sat_shift_limit=(0, 14), val_shift_limit=(3, 0))
])

def loadAndCropCenterResizeCV2(img, newSize):
    #img = cv.imread(imgPath)
    width, height, ______ = img.shape
    if width == height:
        return cv.resize(img, newSize)
    length = min(width, height)
    left = (width - length) // 2
    top = (height - length) // 2
    right = (width + length) // 2
    bottom = (height + length) // 2
    return cv.resize(img[left:right, top:bottom, :], newSize)

def clahe_resize(impath):
  img = cv.imread(impath)

  # Convert to RGB Color Space
  image = cv.cvtColor(img, cv.COLOR_BGR2RGB)

  # Apply Albumentations Augmentation (CLAHE)
  transformed = transform(image=image)
  # Get result
  transformed_image = transformed["image"]
  # Convert back to BGR colorspace
  transformed_image_BGR = cv.cvtColor(transformed_image, cv.COLOR_BGR2RGBA)

  #clahe = cv.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
  #L, a, b = cv.split(cv.cvtColor(img, cv.COLOR_BGR2Lab))
  #cl1 = clahe.apply(L)
  #eq_image = cv.cvtColor(cv.merge([cl1, a, b]), cv.COLOR_Lab2BGR)

  eq_image = loadAndCropCenterResizeCV2(transformed_image_BGR, (250, 250))


  
  cv.imwrite(impath,eq_image)




In [ ]:
# References: https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
def resize_and_letter_box(image, rows, cols):
    """
    Letter box (black bars) a color image (think pan & scan movie shown 
    on widescreen) if not same aspect ratio as specified rows and cols. 
    :param image: numpy.ndarray((image_rows, image_cols, channels), dtype=numpy.uint8)
    :param rows: int rows of letter boxed image returned  
    :param cols: int cols of letter boxed image returned
    :return: numpy.ndarray((rows, cols, channels), dtype=numpy.uint8)
    """
    image_rows, image_cols = image.shape[:2]
    row_ratio = rows / float(image_rows)
    col_ratio = cols / float(image_cols)
    ratio = min(row_ratio, col_ratio)
    image_resized = cv.resize(image, dsize=(0, 0), fx=ratio, fy=ratio)
    letter_box = np.zeros((int(rows), int(cols), 3))
    row_start = int((letter_box.shape[0] - image_resized.shape[0]) / 2)
    col_start = int((letter_box.shape[1] - image_resized.shape[1]) / 2)
    letter_box[row_start:row_start + image_resized.shape[0], col_start:col_start + image_resized.shape[1]] = image_resized
    return letter_box

# References https://note.nkmk.me/en/python-opencv-hconcat-vconcat-np-tile/
def hconcat_resize_min(im_list, interpolation=cv.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv.hconcat(im_list_resize)


def fuseImages(impath1, impath2, fusedPathName):
  img1 = cv.imread(impath1)
  img2 = cv.imread(impath2)


  #fusion = "SUM"
  if (fusionMethod == "SUM"):
    img1Width = img1.shape[1]
    img1Height = img1.shape[0]
    print("Image1 width:" + str(img1Width) + " - Image1 height: " + str(img1Height))

    img2Width= img2.shape[1]
    img2Height = img2.shape[0]
    print("Image2 width:" + str(img1Width) + " - Image2 height: " + str(img1Height))

    if img1Width > img2Width:
      img2 = resize_and_letter_box(img2, img1Height, img1Width)
    else:
      img1 =  resize_and_letter_box(img1, img2Height, img2Width)
    

    # img1 = img1.reshape(img2.shape)
    fused = img1 + img2 #element wise sum

  elif (fusionMethod == "CONCAT"):
    fused = hconcat_resize_min([img1, img2])


   

  cv.imwrite(fusedPathName, fused)

all_paths = []

#images enhancing and resizing (takes a while ~19min)
for imagePath in sorted(glob.glob("ODIR-5K/Training Images/*.jpg")):
  # Check if there is a left and right eye image  
  imagePathSplit = imagePath.split("/")
  imageName = imagePathSplit[len(imagePathSplit) - 1]

  imageNameSplit = imageName.split("_")
  imageIndex = imageNameSplit[0]
  leftOrRight = imageNameSplit[1].split("-")[0]
  #print("Left or right: " + leftOrRight)
  if (leftOrRight == "right"):
    # Since we're sorting all paths, left always occurs before right and we process both left/right images when we find the left eye
    #print("Removing right: " + imagePath)
    os.remove(imagePath)
    continue
  
  # Just incase, so we don't fuse more than once in the event this cell is ran multiple times
  if (leftOrRight == "fused"):
    continue

  imagePairs = glob.glob("ODIR-5K/Training Images/" + imageIndex + "_*.jpg")
  #print(imagePairs)
  if (len(imagePairs) != 2):
    # Skip images that don't have a corresponding left and right eye image
    os.remove(imagePath)
    print("Removing cause there isint 2 images: " + imagePath)
    continue
  
  bothEyesDiseases = set()
  for eyeImage in imagePairs:
    # Get all the diseases of both eyes

    # Get filename at end of path
    eyeImagePathSplit = imagePath.split("/")
    eyeImageFilename = eyeImagePathSplit[len(eyeImagePathSplit) - 1]

    # Remove file extension
    base = eyeImageFilename.split(".")[0]

    diseasesString = base.split("-")[1]
    diseases = diseasesString.split("&")
    for disease in diseases:
      bothEyesDiseases.add(disease)

  # Create the new filename
  fusedFileName = imageIndex + "_" + "fused" + "-"
  numDiseases = len(bothEyesDiseases)
  count = 0
  for disease in bothEyesDiseases:
    fusedFileName += disease
    count += 1
    if count != numDiseases:
      fusedFileName += "&"
  
  fusedFileName += ".jpg"
  #print(fusedFileName)
  fusedFilePath = "ODIR-5K/Training Images/" + fusedFileName

  # Now we can actually fuse the images together
  if (fusionMethod != "CONCAT"):
    fuseImages(imagePairs[0], imagePairs[1], fusedFilePath)
    clahe_resize(fusedFilePath)
  else:
    # When concatenating, we apply enhancements and resize the images before concatenating them
    clahe_resize(imagePairs[0])
    clahe_resize(imagePairs[1])
    fuseImages(imagePairs[0], imagePairs[1], fusedFilePath)

  

  all_paths.append(fusedFilePath)

  # Delete the left eye image image
  # TODO: Just make a new folder for fused images?..
  os.remove(imagePath)



# print("All paths: " + str(all_paths))

#images enhancing and resizing (takes a while ~19min)
#all_paths = []
#for element in sorted(glob.glob("ODIR-5K/Training Images/*.jpg")):
#  all_paths.append(element)
#  clahe_resize(element)

# Set a random seed so we always use the same validation set (which is randomly sampled from dataset)
random.seed(7)

#creating validation set
# num_to_select = 1950
num_to_select = int( 0.3 * len(all_paths))                         
list_of_random_items = random.sample(sorted(all_paths), num_to_select)
for element in list_of_random_items:
  p = element.split("/")
  os.replace(element, "ODIR-5K/Validation_Images/"+p[-1])

In [ ]:
!rm -rf ODIR-5K/Testing\ Images/
!rm -rf ODIR-5K/data.xlsx
!zip -r ODIR-5K_contrast.zip ODIR-5K/

In [8]:
from google.colab import drive
drive.mount('/amd/')

Mounted at /amd/


In [11]:
!cp -r ODIR-5K_contrast.zip /amd/My\ Drive/ML490

In [12]:
def show_class_distribution():
  N = 0
  D=0
  G=0
  C=0
  A=0
  H=0
  M=0
  O = 0
  X = 0
  for element in glob.glob("ODIR-5K/Training Images/*.jpg"):
    img_name = element.split("/")[-1]
    img_label = img_name.split("-")
    if img_label[-1] == "N.jpg":
      N += 1
    elif img_label[-1] == "D.jpg":
      D +=1
    elif img_label[-1] == "G.jpg":
      G +=1
    elif img_label[-1] == "C.jpg":
      C +=1
    elif img_label[-1] == "A.jpg":
      A +=1
    elif img_label[-1] == "H.jpg":
      H +=1
    elif img_label[-1] == "M.jpg":
      M +=1
    elif img_label[-1] == "O.jpg":
      O +=1
    elif "&" in img_label[-1]:
      X +=1  
  print(N, D, G, C, A, H, M, O, X)
show_class_distribution()

980 470 69 101 82 41 82 234 186
